<a href="https://colab.research.google.com/github/JackYoustra/CVEPitch/blob/master/CVEs_and_stonks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install wget gsutil beautifulsoup4 autokeras tf-nightly # yeye hype train maybe not neede tho

     |████████████████████████████████| 2.3MB 5.0MB/s 
     |████████████████████████████████| 71kB 9.1MB/s 
     |████████████████████████████████| 400.2MB 39kB/s 
     |████████████████████████████████| 174kB 50.2MB/s 
     |████████████████████████████████| 61kB 8.7MB/s 
     |████████████████████████████████| 61kB 7.6MB/s 
     |████████████████████████████████| 26.6MB 1.3MB/s 
     |████████████████████████████████| 92.5MB 112kB/s 
     |████████████████████████████████| 582.5MB 26kB/s 
     |████████████████████████████████| 61kB 9.1MB/s 
     |████████████████████████████████| 17.3MB 468kB/s 
     |████████████████████████████████| 5.4MB 37.1MB/s 
     |████████████████████████████████| 25.3MB 1.7MB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
     |████████████████████████████████| 3.3MB 34.2MB/s 
     |████████████████████████████████| 450kB 39.3MB/s 
     |████████████████████████████████| 3.8MB 34.5MB/s 
     |████████████████████████████████| 112kB 43.2MB/s 
   

In [0]:
from pathlib import Path
resources = Path("./resources")
# get severity statistics as well
severity_path = Path("./resources/CVE_severity_list")

In [0]:
import wget

#download stuff here

if not resources.exists():
  resources.mkdir()

if not severity_path.exists():
    severity_path.mkdir()

base_url = "https://nvd.nist.gov/feeds/json/cve/1.1/"
for year in range(2002, 2020):
    path_ext = "nvdcve-1.1-" + str(year) + ".json.gz"
    dest_path = severity_path / path_ext
    if not dest_path.exists():
        wget.download(base_url + path_ext, str(dest_path.absolute()))
    

In [4]:
# pull in the severity stuff too
# should be easily parallelizeable

import gzip as gz
import json

cve_items = []
for cve_year in severity_path.iterdir():
    with gz.GzipFile(cve_year, "r") as zipfile:
        data = json.loads(zipfile.read())
        cve_items.extend(data['CVE_Items'])

print(len(cve_items))


132446


In [5]:
print(cve_items[0].keys())
import json
jsjs = json.dumps(cve_items[0], sort_keys=True, indent=4)

print(jsjs)

dict_keys(['cve', 'configurations', 'impact', 'publishedDate', 'lastModifiedDate'])
{
    "configurations": {
        "CVE_data_version": "4.0",
        "nodes": [
            {
                "cpe_match": [
                    {
                        "cpe23Uri": "cpe:2.3:o:linux:linux_kernel:2.2.7:*:*:*:*:*:*:*",
                        "vulnerable": true
                    },
                    {
                        "cpe23Uri": "cpe:2.3:o:linux:linux_kernel:2.4.0:*:*:*:*:*:*:*",
                        "vulnerable": true
                    },
                    {
                        "cpe23Uri": "cpe:2.3:o:linux:linux_kernel:2.4.1:*:*:*:*:*:*:*",
                        "vulnerable": true
                    },
                    {
                        "cpe23Uri": "cpe:2.3:o:linux:linux_kernel:2.4.2:*:*:*:*:*:*:*",
                        "vulnerable": true
                    },
                    {
                        "cpe23Uri": "cpe:2.3:o:linux:linux_kernel

In [0]:
## So CPEs seem like a cool thing
# We should do something with that

cpe_path = resources / "cpe_2-3.xml.zip"
if not cpe_path.exists():
  wget.download("https://nvd.nist.gov/feeds/xml/cpe/dictionary/official-cpe-dictionary_v2.3.xml.zip", str(cpe_path.absolute()))



In [9]:
import zipfile
import xml.etree.ElementTree as ET
tree = None
with zipfile.ZipFile(cpe_path,"r") as zip_ref:
  for filename in zip_ref.namelist():
    with zip_ref.open(filename) as f:
      data = f.read()
      tree = ET.fromstring(data)
    break

tree

<Element '{http://cpe.mitre.org/dictionary/2.0}cpe-list' at 0x7f4949598db8>

In [10]:
for cpe_root in tree:
  print(len(cpe_root))
  print(cpe_root[0:4])

4
[<Element '{http://cpe.mitre.org/dictionary/2.0}product_name' at 0x7f4949598ea8>, <Element '{http://cpe.mitre.org/dictionary/2.0}product_version' at 0x7f4949598ef8>, <Element '{http://cpe.mitre.org/dictionary/2.0}schema_version' at 0x7f4949598f48>, <Element '{http://cpe.mitre.org/dictionary/2.0}timestamp' at 0x7f49495a3048>]
3
[<Element '{http://cpe.mitre.org/dictionary/2.0}title' at 0x7f49495a3228>, <Element '{http://cpe.mitre.org/dictionary/2.0}references' at 0x7f49495a32c8>, <Element '{http://scap.nist.gov/schema/cpe-extension/2.3}cpe23-item' at 0x7f4949598a48>]
3
[<Element '{http://cpe.mitre.org/dictionary/2.0}title' at 0x7f4949598958>, <Element '{http://cpe.mitre.org/dictionary/2.0}references' at 0x7f4949598908>, <Element '{http://scap.nist.gov/schema/cpe-extension/2.3}cpe23-item' at 0x7f49495a34a8>]
3
[<Element '{http://cpe.mitre.org/dictionary/2.0}title' at 0x7f49495a3548>, <Element '{http://cpe.mitre.org/dictionary/2.0}references' at 0x7f49495a35e8>, <Element '{http://scap.ni

KeyboardInterrupt: ignored

In [0]:
# some sense of what the data even looks like



In [12]:
# okay, great, now we need to get the thing we're trying to predict
# Alex recommends CIBR, so we're rolling with that for now
# Perhaps XSW too, at some point
# I'd LOVE NASDAQ, but nooooo can't have that

!mkdir -p resources/CIBR
!gsutil -m cp gs://alpaca-market-data/CIBR/* resources/CIBR

Copying gs://alpaca-market-data/CIBR/CIBR_minute_data_1999-01-01_EMPTY.csv...
Copying gs://alpaca-market-data/CIBR/CIBR_minute_data_1999-02-01_EMPTY.csv...
Copying gs://alpaca-market-data/CIBR/CIBR_minute_data_1999-03-01_EMPTY.csv...
Copying gs://alpaca-market-data/CIBR/CIBR_minute_data_1999-04-01_EMPTY.csv...
Copying gs://alpaca-market-data/CIBR/CIBR_minute_data_1999-05-01_EMPTY.csv...
Copying gs://alpaca-market-data/CIBR/CIBR_minute_data_1999-08-01_EMPTY.csv...
Copying gs://alpaca-market-data/CIBR/CIBR_minute_data_1999-07-01_EMPTY.csv...
Copying gs://alpaca-market-data/CIBR/CIBR_minute_data_1999-06-01_EMPTY.csv...
Copying gs://alpaca-market-data/CIBR/CIBR_minute_data_1999-09-01_EMPTY.csv...
Copying gs://alpaca-market-data/CIBR/CIBR_minute_data_1999-10-01_EMPTY.csv...
Copying gs://alpaca-market-data/CIBR/CIBR_minute_data_1999-11-01_EMPTY.csv...
Copying gs://alpaca-market-data/CIBR/CIBR_minute_data_1999-12-01_EMPTY.csv...
Copying gs://alpaca-market-data/CIBR/CIBR_minute_data_2000-01-01

In [13]:
# HOWEVER we can get IVV, an ETF which mimics GSPC (maybe should've gone nasdaq oops)
!mkdir -p resources/IVV
!gsutil -m cp gs://alpaca-market-data/IVV/* resources/IVV

Copying gs://alpaca-market-data/IVV/IVV_minute_data_1999-01-01_EMPTY.csv...
Copying gs://alpaca-market-data/IVV/IVV_minute_data_1999-02-01_EMPTY.csv...
Copying gs://alpaca-market-data/IVV/IVV_minute_data_1999-06-01_EMPTY.csv...
Copying gs://alpaca-market-data/IVV/IVV_minute_data_1999-08-01_EMPTY.csv...
Copying gs://alpaca-market-data/IVV/IVV_minute_data_1999-03-01_EMPTY.csv...
Copying gs://alpaca-market-data/IVV/IVV_minute_data_1999-07-01_EMPTY.csv...
Copying gs://alpaca-market-data/IVV/IVV_minute_data_1999-04-01_EMPTY.csv...
Copying gs://alpaca-market-data/IVV/IVV_minute_data_1999-05-01_EMPTY.csv...
Copying gs://alpaca-market-data/IVV/IVV_minute_data_1999-09-01_EMPTY.csv...
Copying gs://alpaca-market-data/IVV/IVV_minute_data_1999-10-01_EMPTY.csv...
Copying gs://alpaca-market-data/IVV/IVV_minute_data_1999-11-01_EMPTY.csv...
Copying gs://alpaca-market-data/IVV/IVV_minute_data_1999-12-01_EMPTY.csv...
Copying gs://alpaca-market-data/IVV/IVV_minute_data_2000-01-01_EMPTY.csv...
Copying gs:/

In [19]:
# nasdaq time
!mkdir -p resources/QQQ
!gsutil -m cp gs://alpaca-market-data/QQQ/* resources/QQQ

!mkdir -p resources/INTC
!gsutil -m cp gs://alpaca-market-data/INTC/* resources/INTC

!mkdir -p resources/MSFT
!gsutil -m cp gs://alpaca-market-data/MSFT/* resources/MSFT

Copying gs://alpaca-market-data/QQQ/QQQ_minute_data_1999-07-01_EMPTY.csv...
Copying gs://alpaca-market-data/QQQ/QQQ_minute_data_1999-06-01_EMPTY.csv...
Copying gs://alpaca-market-data/QQQ/QQQ_minute_data_1999-09-01_EMPTY.csv...
Copying gs://alpaca-market-data/QQQ/QQQ_minute_data_1999-08-01_EMPTY.csv...
Copying gs://alpaca-market-data/QQQ/QQQ_minute_data_1999-01-01_EMPTY.csv...
Copying gs://alpaca-market-data/QQQ/QQQ_minute_data_1999-02-01_EMPTY.csv...
Copying gs://alpaca-market-data/QQQ/QQQ_minute_data_1999-04-01_EMPTY.csv...
Copying gs://alpaca-market-data/QQQ/QQQ_minute_data_1999-03-01_EMPTY.csv...
Copying gs://alpaca-market-data/QQQ/QQQ_minute_data_1999-05-01_EMPTY.csv...
Copying gs://alpaca-market-data/QQQ/QQQ_minute_data_1999-10-01_EMPTY.csv...
Copying gs://alpaca-market-data/QQQ/QQQ_minute_data_1999-11-01_EMPTY.csv...
Copying gs://alpaca-market-data/QQQ/QQQ_minute_data_1999-12-01_EMPTY.csv...
Copying gs://alpaca-market-data/QQQ/QQQ_minute_data_2000-01-01_EMPTY.csv...
Copying gs:/

In [14]:
import pandas as pd
stockpath = resources / 'CIBR'

# dataframe time!
# https://stackoverflow.com/a/44514187/998335
cibr_data = pd.DataFrame(columns=['open','high','low','close','volume','millis'])


# https://stackoverflow.com/a/32237949/998335
def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))

for file in stockpath.iterdir():
  # CSV of format timestamp,open,high,low,close,volume,millis with headers
  current = pd.read_csv(file)
  if 'timestamp' in current.columns.values:
    cibr_data = cibr_data.merge(current, how='outer', copy=False, validate=None)

cibr_data.sort_values('millis', inplace=True)
cibr_data.reset_index(drop=True, inplace=True)

cibr_data

,timestamp,open,high,low,close,volume,millis
0,2015-07-07 09:30:00,20.3000,20.3000,20.3000,20.3000,10,1.436276e+09
1,2015-07-07 09:31:00,20.0000,20.0000,20.0000,20.0000,200,1.436276e+09
2,2015-07-07 09:32:00,20.0000,20.0000,20.0000,20.0000,800,1.436276e+09
3,2015-07-07 09:33:00,20.0400,20.0400,20.0400,20.0400,10,1.436276e+09
4,2015-07-07 09:34:00,19.9082,19.9300,19.9082,19.9300,2000,1.436276e+09
...,...,...,...,...,...,...,...
154251,2019-09-13 15:03:00,28.1500,28.1500,28.1300,28.1350,2026,1.568401e+09
154252,2019-09-13 15:04:00,28.1222,28.1400,28.1222,28.1400,2829,1.568401e+09
154253,2019-09-13 15:05:00,28.1400,28.1505,28.1000,28.1350,700,1.568402e+09
154254,2019-09-13 15:06:00,28.1350,28.1400,28.1152,28.1152,1410,1.568402e+09


In [0]:
other_stocks = ['IVV', 'QQQ', 'INTC', 'MSFT']

def compose_close(ticker, nrows=None):
  ivvpath = resources / ticker

  ivvdata = pd.DataFrame(columns=['timestamp','close'])
  for file in ivvpath.iterdir():
    # CSV of format timestamp,open,high,low,close,volume,millis with headers
    current = pd.read_csv(file, nrows=nrows)
    if 'timestamp' in current.columns.values:
      current.drop(current.columns.difference(['timestamp','close']), 1, inplace=True)
      ivvdata = ivvdata.merge(current, how='outer', copy=False, validate=None)
  return ivvdata

In [25]:

final_stock_data = pd.DataFrame(columns=['timestamp'])

for stock in other_stocks:
  table = compose_close(stock, nrows=10)
  final_stock_data = final_stock_data.merge(table, how='outer', on=['timestamp'], copy=False, suffixes=('', '_{}'.format(stock)), validate=None)

final_stock_data

,timestamp,close,close_QQQ,close_INTC,close_MSFT
0,2015-08-03 08:47:00,211.66,NaN,NaN,NaN
1,2015-08-03 09:19:00,211.84,NaN,NaN,NaN
2,2015-08-03 09:20:00,211.86,NaN,NaN,NaN
3,2015-08-03 09:25:00,211.76,NaN,NaN,46.87
4,2015-08-03 09:29:00,211.76,NaN,29.09,NaN
...,...,...,...,...,...
5798,2015-07-01 06:36:00,NaN,NaN,NaN,44.49
5799,2015-07-01 06:45:00,NaN,NaN,NaN,44.55
5800,2015-07-01 06:58:00,NaN,NaN,NaN,44.55
5801,2015-07-01 07:15:00,NaN,NaN,NaN,44.49


In [0]:
# let's do a test
# we're going to need vulnerability stats on times (i.e. vulnerability statistics as an indicator for this stuff)
# RNN because why not
import tensorflow as tf
from datetime import datetime as dt

# create the dataset
# probably a way I can read directly from the json into tf, but for now manual processing
# we want to have the prediction value be the point change (day to day, we'll call it) (regressive)
# and the 

# gets the feature vector from a cve entry
# Current features:
# description: string
# vector: keyword. Example: NETWORK
# complexity: keyword. Example: LOW
# authentication: keyword. Example: NONE
# confidentialityImpact: keyword. Example: NONE
# integrityImpact: keyword. Example: NONE
# availabilityImpact: keyword. Example: PARTIAL
# baseScore: float
# severity: keyword. Example: MEDIUM
# exploitabilityScore: float
# impactScore: float
# obtainAllPrivilege: Boolean
# obtainUserPrivilege: Boolean
# obtainOtherPrivilege: Boolean
# userInteractionRequired: Boolean
def feature_vector(cve):
    main_data = cve['cve']
    description = main_data['description']['description_data'][0]['value']
    impact = cve['impact']['baseMetricV2']
    cvss = impact['cvssV2']
    
    return description, cvss['accessVector'], cvss['accessComplexity'], cvss['authentication'], \
            cvss['confidentialityImpact'], cvss['integrityImpact'], cvss['availabilityImpact'], \
            cvss['baseScore'], impact['severity'], impact['exploitabilityScore'], \
            impact['impactScore'], impact['obtainAllPrivilege'], impact['obtainUserPrivilege'], \
            impact['obtainOtherPrivilege'], impact.get('userInteractionRequired', False)

cve_items = [cve for cve in cve_items if 'baseMetricV2' in cve['impact']]

features = [feature_vector(x) for x in cve_items]

publishedMillis = [dt.strptime(cve['publishedDate'] + "UTC", '%Y-%m-%dT%H:%MZ%Z').timestamp() for cve in cve_items]

timestamps = stock_data['millis'].values


In [18]:
# lets see overall data counts
import matplotlib.pyplot as plt

feature_indices = (7, 9, 10)
fig, axs = plt.subplots(1, len(feature_indices), tight_layout=True)

for i in range(len(feature_indices)):
  index = feature_indices[i]
  for feature in features:
    axs[i].hist([feature[index] for feature in features], bins=10)

fig

KeyboardInterrupt: ignored

Error in callback <function flush_figures at 0x7f4ad00b1158> (for post_execute):


KeyboardInterrupt: ignored

In [0]:
import numpy as np
import math

# TODO: reject if too far different?
def closest_index(timestamp):
  index_candidate = np.searchsorted(timestamps, timestamp, side='left')
  if index_candidate + 1 >= len(timestamps):
    return len(timestamps) - 1
  elif abs(timestamps[index_candidate] - timestamp) < abs(timestamps[index_candidate + 1] - timestamp):
    return index_candidate
  return index_candidate + 1

# now we need to bundle this with what we're trying to predict
# given a CVE, we're trying to predict the movement
# from the closest time we have a signal to interval minutes after
# iff the closest time we have a signal is leq interval minutes away
def synthesize_signal(interval):
  X = []
  y = []
  current_closes = []
  millis = interval * 60 * 1000
  for i in range(len(cve_items)):
    publication = publishedMillis[i]
    # purge the ones that aren't released in interval of any trade
    if publication < timestamps[0] or publication > timestamps[-1]:
      continue
    # find the closest related trade
    current_index = closest_index(publication)
    # and the one for the future
    future_index = closest_index(millis + publication)
    # can't be the same, that'd be a trivial prediction
    assert(current_index < future_index)
    current_trade = stock_data.iloc[current_index]
    future_trade = stock_data.iloc[future_index]
    # our prediction
    current_close = current_trade["close_cibr"]
    change = future_trade["close_cibr"] - current_close
    X.append(features[i])
    y.append(change)
    current_closes.append(current_close)
    
  return X, y, current_closes

# aiya ok so we need to do the same, but this time be smart about it and
# actually take into account the fact that OTHER FACTORS EXIST smh
def synthesize_alpha_signal(interval):
  X = []
  y = []
  current_closes = []
  millis = interval * 60 * 1000
  for i in range(len(cve_items)):
    publication = publishedMillis[i]
    # purge the ones that aren't released in interval of any trade
    if publication < timestamps[0] or publication > timestamps[-1]:
      continue
    # find the closest related trade
    current_index = closest_index(publication)
    # and the one for the future
    future_index = closest_index(millis + publication)
    # can't be the same, that'd be a trivial prediction
    assert(current_index < future_index)
    current_trade = stock_data.iloc[current_index]
    future_trade = stock_data.iloc[future_index]
    # I actually have no idea when this would be true but whatever
    if math.isnan(current_trade["close_ivv"]) or math.isnan(future_trade["close_ivv"]):
      continue
    # our prediction (whether cibr will outperform)
    current_close = current_trade["close_cibr"] - current_trade["close_ivv"]
    change = (future_trade["close_cibr"] - future_trade["close_ivv"]) - \
          current_close
    X.append(features[i])
    y.append(change)
    current_closes.append(current_close)
    
  return X, y, current_closes

In [0]:
# final synthesis

# converts all features into numeric features only so I can be lazy
# blyat this took too long https://stackoverflow.com/a/35487663/998335
def numeric_features_only(X):
  return np.array([np.array([s for s in x if isinstance(s, (float, int, bool))]).astype(float) for x in X])

# day interval
X, y_continuous, current_closes = synthesize_alpha_signal(60*24)

# useful stuff for later
stddev = np.std(y_continuous)
midpoint = np.mean(y_continuous)

X = numeric_features_only(X)
print("Number of samples: " + str(len(X)))

In [0]:
# models, models, models!

from sklearn import svm
from sklearn.linear_model import SGDRegressor, ElasticNet
from sklearn.model_selection import train_test_split
from joblib import Parallel, delayed

experiments = 5

model_metadata = (("SVR", svm.SVR(gamma='scale')), ("SGD", SGDRegressor()), ("ElasticNet", ElasticNet()))

# profitability means run profitability tests instead of just statistical
# accuracy tests
def run_model(model_metadata, X, y, profitability=False):
  name, model = model_metadata
  clf = model
  if profitability:
    sharpe = 0.0 # mean of all orther sharpes
    rfr = 1.90
    for i in range(experiments):
      X_train, X_test, y_train, y_test, current_close_train, current_close_test = train_test_split(X, y, current_closes)
      returns = np.zeros(len(y_test))
      clf.fit(X_train, y_train)
      for i in range(len(X_test)):
        tester = X_test[i].reshape(1, -1)
        actual_value = y_test[i]
        close = current_close_test[i]
        predicted = clf.predict(tester)[0]
        # dumb trading strat
        # first, check if classifications
        if isinstance(actual_value, int):
          # classification
          # if we predict down or up, trade and see how we do after a day
          if predicted == 1:
            # spread grows
            returns[i] = (actual_value - close) / close
          elif predicted == -1:
            returns[i] = (close - actual_value) / close
          else:
            assert predicted == 0
          
        else:
          #TODO just not worth it rn
          assert False
        return_variance = np.var(returns)
        if return_variance > 0:
          total_return = np.sum(returns)
          sharpe += ((total_return - rfr) / return_variance) / experiments
      return name + " has a sharpe of " + str(sharpe) 
    
  else:
    test_acc = 0.0

    for i in range(experiments):
      X_train, X_test, y_train, y_test = train_test_split(X, y)

      # SVR first because hella easy
      test_acc += clf.fit(X_train, y_train).score(X_test, y_test) / experiments

    return name + " has an accuracy of " + str(test_acc)
 
run_model(model_metadata[2], X, y_continuous)
#Parallel(n_jobs = len(model_metadata))(delayed(run_model)(metadata) for metadata in model_metadata)

In [0]:
# classification model
# within a standard deviation is class 0, above and below are 1 and -1

from sklearn import svm
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

# we can def do a better job than this
classification_model_metadata = ( \
                                 ("LinearSVC", svm.LinearSVC()), \
                                 ("Random Forest", RandomForestClassifier()), \
                                 ("AdaBoost", AdaBoostClassifier()) \
                                )

def classify(value):
  if value > midpoint + stddev / 2:
    return 1
  elif value < midpoint - stddev / 2:
    return -1
  return 0

y_classified = [classify(out) for out in y_continuous]

for meta in classification_model_metadata:
  print(run_model(meta, X, y_classified, profitability=True))

In [0]:
import autokeras as ak

classification_error = 0.0

for i in range(experiments):
  X_train, X_test, y_continuous_train, y_continuous_test, y_classified_train, y_classified_test = train_test_split(X, y_continuous, y_classified)

  error_unit = 1 / (experiments + len(y_classified_test))
  # man they don't have their structured data stuff complete
  # gotta actually do it ourself
  # Build model and train.
  automodel = ak.AutoModel(
     inputs=[ak.StructuredDataInput(),
             ak.StructuredDataInput()],
     outputs=[ak.RegressionHead(metrics=['mae']),
              ak.ClassificationHead(loss='categorical_crossentropy',
                                    metrics=['accuracy'])])
  # this sign can't stop me because I can't read
  automodel.fit([X_train, X_train],
                [y_continuous_train, y_classified_train])
  
  # let's just hope classification error follows other error
  for i in range(len(y_classified_test)):
    classified_test = y_classified_test[i]
    tester = X_test[i]
    regressive, classified_empirical = automodel(tester, tester)
    if classified_test != classified_empirical:
      classification_error += error_unit

classification_error